## IFC Furniture & Occupancy Processing (Chair-Based Occupancy Detection)

This module provides a complete workflow for detecting furniture (specifically chairs) inside 
IfcSpaces using an IFC model. Its main purpose is to determine occupancy per space based on the 
number of chairs and then prepare the data for fallback rules such as BR18.

The module includes:

- Extraction of decomposition structure (assembly / sub-elements / standalone)
- Identification of chairs using several heuristics:
  - PredefinedType at the instance level
  - PredefinedType from related furniture types
  - Name/ObjectType keyword matching
- Mapping of contained elements inside each IfcSpace
- Counting chairs per space to infer occupancy
- Application of BR18 fallback logic where no chairs are detected
- ANNEX C category guessing for estimated densities

Each section of the code is structured to work in a pipeline, intended for integration with 
larger IFC analysis workflows.


In [ ]:
from __future__ import annotations
from os import name
from pathlib import Path
from tabnanny import verbose
from typing import Dict, Tuple, Optional, List, Callable, Any
import logging

import ifcopenshell 
import ifcopenshell.util as util
import ifcopenshell.util.element as uel
import ifcopenshell.util.placement as up
import ifcopenshell.geom as geom

import math
from pprint import pprint
from collections import Counter, defaultdict

logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")


## Function: classify_assembly_status()

This function examines an IFC entity and determines its structural decomposition role.

It returns three possible classifications:

- **"sub-element"** → the element belongs to an assembly (it is a child)
- **"assembly"** → the element is an assembly containing other elements (children)
- **"standalone"** → the element is neither part of an assembly nor contains sub-elements

This classification is essential for chair detection because IFC models often break furniture 
down into components (e.g. seat, legs, desk surface).  
By skipping sub-elements, the script avoids counting parts of assemblies as separate chairs.


In [ ]:
def classify_assembly_status(entity):
    """
    Classifies an Ifc element's decomposition role.
    """
    if getattr(entity, "Decomposes", None):
        parent = entity.Decomposes[0].RelatingObject
        return "sub-element", parent, None

    if getattr(entity, "IsDecomposedBy", None):
        children = entity.IsDecomposedBy[0].RelatedObjects
        return "assembly", None, children

    return "standalone", None, None


## Function: _get_furniture_predefined_type()

This helper function retrieves the `PredefinedType` of an IfcFurniture element.  
Since IFC models frequently store this information either:

- directly on the instance, or  
- inside its associated IfcFurnitureType (via IfcRelDefinesByType),

this function checks both locations.

The result is used later to more reliably detect whether a furniture element is a chair.


In [ ]:
def _get_furniture_predefined_type(entity) -> Optional[str]:
    """Return PredefinedType from the instance or its type, if present."""
    Furniture = getattr(entity, "PredefinedType", None)
    if isinstance(Furniture, str) and Furniture:
        return Furniture

    if getattr(entity, "IsTypedBy", None):
        rel = entity.IsTypedBy[0]
        if getattr(rel, "RelatingType", None):
            t = rel.RelatingType
            tp = getattr(t, "PredefinedType", None)
            if isinstance(tp, str) and tp:
                return tp
    return None


## Function: _looks_like_chair()

This function determines whether an IfcFurniture element should be considered a chair.

It combines multiple techniques:
1. **PredefinedType = "CHAIR"** (strongest signal)
2. Searching for keywords such as “CHAIR” or “SEAT” in:
   - Name
   - ObjectType
3. Flexible keyword matching (customizable per project)

Models vary significantly in how they encode furniture, so using multiple detection methods 
ensures higher reliability.


In [ ]:
def _looks_like_chair(entity, chair_keywords: Tuple[str, ...]) -> bool:
    """Heuristics to decide if an IfcFurniture element is a chair."""
    if not entity.is_a("IfcFurniture"):
        return False

    ptype = _get_furniture_predefined_type(entity)
    if isinstance(ptype, str) and ptype.upper() == "CHAIR":
        return True

    name = (getattr(entity, "Name", "") or "").upper()
    objtype = (getattr(entity, "ObjectType", "") or "").upper()
    hay = f"{name} {objtype}"

    keywords_upper = [kw.upper() for kw in chair_keywords]
    for kw in keywords_upper:
        if kw in hay:
            return True

    return False


## Count Chairs per Space  
This section explains the function that detects and counts all chairs inside each `IfcSpace` by examining the spatial containment relationships of the IFC model.  
It identifies furniture elements classified as chairs either via `PredefinedType` or by keyword matching (e.g., “chair”, “stol”).  
It supports optional exclusion of assembly subcomponents to prevent double counting.  

The output of this step is a list where each item represents one space and contains:  
- Space ID, Name, Number  
- A list of detected chair objects  
- Total chair count  
- Flags used later for occupant estimation  



In [ ]:
def count_chairs_by_space(
    arch_model,
    chair_keywords=("chair", "stol", "seat"),
    exclude_assembly_children=True,
    verbose=False,
):
    """
    Count chairs per IfcSpace using spatial containment relations.
    """
    from collections import defaultdict
    from typing import Dict, List, Any

    spaces = arch_model.by_type("IfcSpace")
    rels = arch_model.by_type("IfcRelContainedInSpatialStructure")

    contained: Dict[int, List] = defaultdict(list)
    for rel in rels:
        sp = rel.RelatingStructure
        if sp and sp.is_a("IfcSpace"):
            contained[sp.id()].extend(rel.RelatedElements or [])

    results = []
    if verbose:
        print(f"Total Spaces: {len(spaces)}")

    for sp in spaces:
        sp_id = sp.id()
        name = getattr(sp, "Name", None)
        space_name = name.strip() if (name and name.strip()) else "(Unnamed Space)"
        space_number = getattr(sp, "LongName", None) or getattr(sp, "Number", None)

        chairs_in_space = []
        seen = set()

        for el in contained.get(sp_id, []):
            if not el.is_a("IfcFurniture"):
                continue
            if exclude_assembly_children and _is_assembly_child(el):
                continue
            if not _looks_like_chchair(el, chair_keywords):
                continue

            gid = getattr(el, "GlobalId", None)
            if gid and gid in seen:
                continue
            seen.add(gid)

            chairs_in_space.append({
                "id": el.id(),
                "GlobalId": gid,
                "Name": getattr(el, "Name", None),
                "PredefinedType": _get_furniture_predefined_type(el),
            })

        entry = {
            "space_id": sp_id,
            "space_name": space_name,
            "space_number": space_number,
            "chair_count": len(chairs_in_space),
            "chairs": chairs_in_space,
            "inferred_occupants": len(chairs_in_space),
            "had_any_chairs": len(chairs_in_space) > 0,
        }

        results.append(entry)

    return results


## Apply BR18 Fallback Occupancy  
This part processes the output of the chair-counting function and assigns final occupancy values.  
If a space contains chairs, the chair count is used directly as occupant count.  
If not, the BR18 fallback estimation function is applied (density or other DS/BR18 rules).  

The result is a new list where each space gets:  
- `final_occupants` → number of people  
- `occupant_source` → “chairs” or “BR18_fallback”  


In [ ]:
def apply_br18_occupancy(
    spaces_with_chairs,
    *,
    br18_fallback,
    verbose=True,
):
    results = []

    for space in spaces_with_chairs:
        chair_count = space.get("chair_count", 0)

        if chair_count > 0:
            final_occupants = chair_count
            source = "chairs"
        else:
            final_occupants = br18_fallback(space)
            source = "BR18_fallback"

        entry = dict(space)
        entry["final_occupants"] = final_occupants
        entry["occupant_source"] = source

        results.append(entry)

    return results


## Annex C Occupant Density Table  
This table contains default occupant densities (m²/person) based on the DS/EN Annex C guidance.  
It is used when a space has no chairs and requires a fallback estimate for occupancy.  
Categories include office types, classrooms, residential units, meeting rooms, and retail spaces.  


In [ ]:
ANNEX_C_OCCUPANT_DENSITY = {
    "office_landscaped": 17.0,
    "office_single": 10.0,
    "meeting_room": 2.0,
    "department_store": 17.0,
    "classroom": 5.4,
    "kindergarten": 3.8,
    "detached_house": 42.5,
    "residential_apartment_retired": 28.3,
}
